In [134]:
import autogen
config_list = autogen.config_list_from_json(
    "oai_config_list", 
    filter_dict={
        "model": ["gpt-3.5-turbo-16k"]
        })

llm_config_editor = { 
    "timeout": 400,
    "seed": 88,
    "temperature": 0,   # editor 特性需要。
    "config_list": config_list
}

llm_config_writer = { 
    "timeout": 400,
    "seed": 88,
    "temperature": 0.9,   # writer 特性需要，提高隨機性，利於創意寫作。
    "config_list": config_list
}

In [135]:
# 群聊指 > 2個 agents時(user_proxy、assistant)
# 較多的智能體，應該可以解決更複雜任務的假設前提下。可以如此嘗試。
# 為避免陷入無限循環，groupchat 的 "max_round= 10" 一定要設置。

In [136]:
# 主要編寫者
writer = autogen.AssistantAgent(
    name = "Writer",  # 可以跟物件名稱不同
    llm_config = llm_config_writer,
    system_message = """Your task is to write an article on a given subject and to rewrite it after receiving feedback from the editor. 
    You should always use traditional chinese to express. Regardless of whether the prompt word is in Traditional Chinese or not.
    Even the Adjustment "The article should be written in English, not in Chinese." you should still using traditional chinese.
    Add a new line with the content of "終止, TERMINATE" at the end after you create a correct version of the article.
    """
)
# 編輯，會就 writer 編寫的內容給予改進建議，並檢查 Editor 是否產生幻覺。
# 多了這一關，會比一般 gpt 聊天機器人減少幻覺產生的機會，而更顯得可靠。
editor = autogen.AssistantAgent(
    name = "Editor",  # 可以跟物件名稱不同
    llm_config = llm_config_editor,
    system_message= """
    Your task is to correct the article submitted by the writer.
    Check if the information is accurate and complete.
    You should not rewrite the article, nor should you repeat the content provided by the editor; just create a list of adjustments to be made.
    Don't ask to write the article in English.
    Do not provide the Adjustment "The article should be written in English, not in Chinese."
    """
)
user_proxy = autogen.UserProxyAgent(
    name = "user_proxy",
    system_message = "A human admin",
    human_input_mode="NEVER",  # NEVER, TERMINATE, ALWAYS
    max_consecutive_auto_reply = 5,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config = {
        "work_dir": "code",
        "use_docker": False,
    }
    # llm_config = llm_config  # user_proxy 似乎可以不需要設置
)


In [137]:
groupchat = autogen.GroupChat(
    agents = [user_proxy, writer, editor],
    messages = [], # 建立一個訊息列表，用來儲存群聊對話的訊息
    max_round= 8, # 群料很重要，設定最大輪數, 防止 editor 不知道何時結束
    # auto, manual, random, round_robin 共4種選項 
    # auto 表示由 LLM 決定，round_robin 表示依 agents 順序輪流選擇下一個發言者
    speaker_selection_method = "auto",  
)

manager = autogen.GroupChatManager(
    groupchat = groupchat, 
    llm_config = llm_config_editor  # 遠則這個比較穩定
)

In [138]:
# Write an article about AI cutting-edge technology development trends in year 2024.
# Write an article about large language model.
user_proxy.initiate_chat(
    manager,
    message = "Write an article about AI cutting-edge technology development trends in year 2024."
)

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


user_proxy (to chat_manager):

Write an article about AI cutting-edge technology development trends in year 2024.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


Writer (to chat_manager):

人工智慧（AI）技術一直在快速發展，為我們帶來了許多驚人的創新和進步。到2024年，我們可以預見AI技術將會迎來更多的突破和進展。讓我們來看看未來五年內AI領域的發展趨勢。

首先，AI在機器學習和深度學習方面的發展將會更加成熟和強大。機器學習是AI的核心，它可以使機器通過學習和訓練從大量的數據中提取模式和知識。隨著技術的不斷進步和數據量的不斷增加，機器學習將變得更加智能和準確。

其次，AI在自然語言處理（NLP）方面的發展將帶來更多的應用場景。NLP是一個研究如何使機器理解和處理人類語言的領域。隨著AI技術的發展，我們可以預見到在2024年，機器將更加準確地理解和回答人類的問題，甚至能夠進行更加自然和流暢的對話。

再者，AI在醫療領域的應用將會更加廣泛。AI可以通過分析和處理醫學影像數據、病歷記錄等信息，幫助醫生做出更準確的診斷和治療方案。這將大大提高醫療質量和效率，拯救更多的生命。

此外，AI在自動駕駛領域的發展也值得我們期待。自動駕駛技術的崛起將改變我們的交通方式和城市生活。在2024年，我們可以預見到越來越多的車輛將具備自動駕駛功能，大大減少交通事故的發生，提高交通效率，同時節省時間和能源。

最後，AI在教育領域的應用也將繼續增加。AI可以根據學生的學習風格和需求提供個性化的教學內容和建議，幫助學生更高效地學習。同時，AI還可以在教學過程中提供早期預警和監控，幫助教師及時了解學生的學習情況和困難，提供更好的幫助和指導。

總的來說，到2024年，AI技術將會在各個領域取得重大突破和進展。人工智慧將成為我們生活中不可或缺的一部分，為我們帶來更多的便利和改變。讓我們期待這個AI發展的未來！

終止, TERMINATE

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


...

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


Editor (to chat_manager):

Adjustments:
- The article should include a brief introduction about AI and its current state of development.
- The article should provide specific examples and evidence to support the predicted trends in AI development in 2024.
- The article should mention any potential challenges or limitations that may arise in the development of AI technology.
- The article should conclude with a summary of the potential impact of AI on various industries and society as a whole.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


Editor (to chat_manager):

人工智能（AI）是一项快速发展的技术，正在以惊人的速度改变我们的世界。它利用算法和大数据分析，模仿人类的智能和学习能力，以解决各种复杂的问题。AI已经在许多领域取得了显著的成果，如机器学习、自然语言处理和图像识别等。当前， AI正处于高速发展阶段，研究人员和工程师们正在不断推动其前进。

到2024年，AI将迎来更多的发展和进步。其中一个重要的趋势是机器学习和深度学习技术的成熟和强大化。这些技术在各个领域中发挥着重要作用。例如，在医疗领域，AI算法已经能够分析医学影像，并提供精确的诊断。到2024年，随着更多数据的积累和算法的改进，AI在医疗领域中的应用将更加广泛，有助于提高诊断准确性和治疗效果。

另一个重要的趋势是自然语言处理（NLP）技术的发展。NLP使机器能够理解和处理人类语言。目前，NLP已经应用到了聊天机器人、智能助手等各种应用中。到2024年，我们可以预见AI在NLP领域的发展将带来更多智能化的语音识别和翻译技术。这将使得人机交互更加自然和便捷。

然而，AI的发展也面临一些挑战和限制。一个重要的挑战是数据隐私和安全。AI需要大量的数据来进行学习和训练，但同时也需要保护个人隐私。另一个限制是技术和资源的限制。AI的发展需要大量的研发成本和专业人才。缺乏足够的资源可能会限制其发展的速度和规模。

尽管面临一些挑战，AI的发展对各个行业和整个社会都有巨大的潜力影响。AI技术带来的自动化和智能化将提高生产效率，改善产品和服务质量。AI还可以应用于交通、能源和环境等领域，推动可持续发展。然而，我们也需要关注AI伦理和社会影响的问题，确保其发展符合人类利益和价值观。

总之，到2024年，AI技术将继续取得重大突破和进展。机器学习和NLP技术的成熟将带来更多应用场景和智能化的产品和服务。然而，我们也需要谨慎看待AI的发展，确保其在人类和社会的长远利益中发挥积极的作用。

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Write an article about AI cutting-edge technology development trends in year 2024.', 'role': 'assistant'}, {'content': '人工智慧（AI）技術一直在快速發展，為我們帶來了許多驚人的創新和進步。到2024年，我們可以預見AI技術將會迎來更多的突破和進展。讓我們來看看未來五年內AI領域的發展趨勢。\n\n首先，AI在機器學習和深度學習方面的發展將會更加成熟和強大。機器學習是AI的核心，它可以使機器通過學習和訓練從大量的數據中提取模式和知識。隨著技術的不斷進步和數據量的不斷增加，機器學習將變得更加智能和準確。\n\n其次，AI在自然語言處理（NLP）方面的發展將帶來更多的應用場景。NLP是一個研究如何使機器理解和處理人類語言的領域。隨著AI技術的發展，我們可以預見到在2024年，機器將更加準確地理解和回答人類的問題，甚至能夠進行更加自然和流暢的對話。\n\n再者，AI在醫療領域的應用將會更加廣泛。AI可以通過分析和處理醫學影像數據、病歷記錄等信息，幫助醫生做出更準確的診斷和治療方案。這將大大提高醫療質量和效率，拯救更多的生命。\n\n此外，AI在自動駕駛領域的發展也值得我們期待。自動駕駛技術的崛起將改變我們的交通方式和城市生活。在2024年，我們可以預見到越來越多的車輛將具備自動駕駛功能，大大減少交通事故的發生，提高交通效率，同時節省時間和能源。\n\n最後，AI在教育領域的應用也將繼續增加。AI可以根據學生的學習風格和需求提供個性化的教學內容和建議，幫助學生更高效地學習。同時，AI還可以在教學過程中提供早期預警和監控，幫助教師及時了解學生的學習情況和困難，提供更好的幫助和指導。\n\n總的來說，到2024年，AI技術將會在各個領域取得重大突破和進展。人工智慧將成為我們生活中不可或缺的一部分，為我們帶來更多的便利和改變。讓我們期待這個AI發展的未來！\n\n終止, TERMINATE', 'name': 'Writer', 'role': 'user'}, {'content': '- The article should be written in E